In [1]:
import tensorflow as tf
import numpy as np
import os
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
import sys

## Load Dataset

In [3]:
def string_to_float_list(row):
    row["pixels"] = np.array([float(string) for string in row["pixels"].split()])

    return row

In [4]:
def df_to_tensor(series):
    return np.concatenate(series.values).reshape(len(series),len(series[0]))

In [5]:
dataset = pd.read_csv("~/data/emotion_reco/face_dataset.csv",sep="|")
dataset = dataset.apply(lambda x: string_to_float_list(x),axis=1)

In [6]:
x = df_to_tensor(dataset["pixels"])
y = dataset["emotion"].values

## Define Network

In [7]:
def create_conv_layer(inputs,filters,kernels,padding,strides,pools,name,mode):
    #print(inputs,filters,kernels,padding,strides,pools,name,mode)
    conv_layer = tf.contrib.layers.conv2d(
        inputs=inputs,
        num_outputs=filters,
        kernel_size=kernels,
        padding=padding,
        stride=1,
        activation_fn=tf.nn.relu,
        normalizer_fn=tf.contrib.layers.batch_norm
    )
    
    #batch_norm = tf.contrib.layers.batch_norm(
    #    inputs=conv_layer)
    
    relu_act = tf.nn.relu(
        features=conv_layer)
    
    dropout = tf.layers.dropout(
        inputs=relu_act,
        #rate=0.4,
        training = mode == tf.estimator.ModeKeys.TRAIN)
    
    pooling = tf.layers.max_pooling2d(
        inputs=dropout,
        pool_size=pools,
        strides=strides,
        padding=padding)
    
    return pooling

In [8]:
def create_fc_layer(inputs,neurons,mode):
    fc_layer = tf.layers.dense(
        inputs=inputs,
        units=neurons
    )
    
    batch_norm = tf.layers.batch_normalization(inputs=fc_layer)
    
    dropout = tf.layers.dropout(
        inputs=batch_norm,
        #rate=0.4,
        training=mode == tf.estimator.ModeKeys.TRAIN)
    
    relu_act = tf.nn.relu(
        features=dropout)

    return relu_act

In [9]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.

    #tf_split = tf.string_split(features,delimiter=' ').values
    #features_split = tf.cast(tf.reshape(tf_split,[48*48]),dtype=tf.float32)
    #features_split = tf.cast(features,dtype=tf.float32)

    
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [10]:
def eval_input_fn(features, labels=None, batch_size=None):
    """An input function for evaluation or prediction"""
    
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)
    
    # Convert inputs to a tf.dataset object.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [11]:
def dense_block(inputs, size, filters, kernel, padding="same", activation=tf.nn.relu, name="denseblock"):
    
    with tf.variable_scope(name) as scope:
        inp = inputs

        for i in range(1, size+1):
            # each layer in the block has batch norm -> relu -> 1x1conv -> batch norm -> relu -> conv

            dense_layer = tf.layers.batch_normalization(inputs=inp)
            dense_layer = tf.nn.relu(dense_layer)
            
            dense_layer = tf.layers.conv2d(inputs=dense_layer,
                                          filters=4*filters,
                                          kernel_size=[1,1],
                                          activation=activation,
                                          #use_bias=False,
                                          padding=padding,
                                          name="bottleneck-layer-"+str(i)
                                          )
            
            dense_layer = tf.layers.batch_normalization(inputs=inp)
            dense_layer = tf.nn.relu(dense_layer)
            
            dense_layer = tf.layers.conv2d(inputs=dense_layer,
                                          filters=filters,
                                          kernel_size=kernel,
                                          activation=activation,
                                          #use_bias=False,
                                          padding=padding,
                                          name="conv-layer-"+str(i)
                                          )
            # concatenate output of last layer with the output of all previous ones
            # inp should contain the concatenation of all ouputs so far
            inp = tf.concat([inp, dense_layer], 3)
        
    return inp

In [12]:
def dense_transition(inputs, filters, activation=tf.nn.relu, name="transition-layer"):
    
    with tf.variable_scope(name) as scope:
        # each transition is batch norm -> relu -> conv -> pooling
    
        transition = tf.layers.batch_normalization(inputs=inputs)
        transition = tf.nn.relu(transition)
        
        transition = tf.layers.conv2d(inputs=transition,
                                          filters=filters,
                                          kernel_size=[1 ,1],
                                          activation=activation,
                                          #use_bias=False,
                                          padding="same",
                                          name="bottleneck-"
                                          )
        
        transition = tf.layers.average_pooling2d(inputs=transition, pool_size=[2,2], strides=2)
        
    return transition

In [13]:
def cnn_dense_model_fn(features, labels, mode):
    """Model function for CNN."""
    filters = 6
    kernel = [3, 3]
    compression_factor = 0.5
    
    n_fc = [500]
    
    dense_sizes = [6]
    dense_blocks = len(dense_sizes)
    
    # Input Layer
    input_layer = tf.reshape(tf.cast(features,tf.float32), [-1, 48, 48, 1])
    #one_hot_labels = tf.one_hot(labels,7)
    
    
    # initial conv layer
    in_layer = create_conv_layer(inputs=input_layer,
                                  filters=filters,
                                  kernels=kernel,
                                  padding="same",
                                  strides=2,
                                  pools=[2,2],
                                  name="conv-1",
                                  mode=mode)    
    
    
    # create denseblocks with the correct depth
    # follow them with a transition layer(unless its the last denseblock)
    # if last denseblock then just do normal avergae pooling
    for i, size in zip(range(1, dense_blocks+1), dense_sizes):
        denseblock = dense_block(inputs=in_layer, 
                                  size=size, 
                                  filters=filters, 
                                  kernel=kernel, 
                                  name="denseblock-"+str(i))

        if i != dense_blocks:
            in_layer = dense_transition(inputs=denseblock, 
                                       filters=int(int(denseblock.shape[3])*compression_factor), 
                                       name="transition-layer-"+str(i))

        else:
            # final "transition" layer
            last_layer_dense = tf.layers.average_pooling2d(inputs=denseblock, pool_size=[2,2], strides=2)
        
     
    # get the amount of total inputs from last layer
    dense_units_output = last_layer_dense.shape[1] * last_layer_dense.shape[2] * last_layer_dense.shape[3]
        
    fc_layer = tf.reshape(last_layer_dense, [-1, dense_units_output])
    
    # create the last fully connected layers
    for n in n_fc:
        fc_layer = create_fc_layer(
            inputs=fc_layer,
            neurons=n,
            mode=mode
        )
        
        
    logits = tf.layers.dense(inputs=fc_layer, units=7)
    
    softmax = tf.nn.softmax(logits, name="softmax_tensor")

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": softmax
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## TRAINING

In [14]:
emotion_classifier_dense = tf.estimator.Estimator(
    model_fn=cnn_dense_model_fn, model_dir="tmp/convnet_dense_model-2")


tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=2000)

tf.logging.set_verbosity(tf.logging.INFO)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_task_type': 'worker', '_save_summary_steps': 100, '_is_chief': True, '_num_worker_replicas': 1, '_tf_random_seed': None, '_model_dir': 'tmp/convnet_dense_model-2', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8632684f60>, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_master': '', '_service': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_session_config': None, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_task_id': 0}


In [15]:
emotion_classifier_dense.train(
    input_fn=lambda:train_input_fn(
        x,
        y,
        128),
    steps=10*35000,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into tmp/convnet_dense_model-2/model.ckpt.
INFO:tensorflow:probabilities = [[0.11149357 0.30097237 0.09129142 0.07490344 0.1646568  0.14742936
  0.10925303]
 [0.1253486  0.05025465 0.18110244 0.07666513 0.24111703 0.27413097
  0.05138117]
 [0.05504357 0.4922126  0.01534976 0.15510777 0.12935431 0.14131647
  0.01161556]
 [0.07070283 0.0614652  0.11021775 0.40702826 0.15010577 0.07749642
  0.12298378]
 [0.14462624 0.2839712  0.05617501 0.12153297 0.29091284 0.07010721
  0.03267457]
 [0.00288071 0.00765647 0.01077637 0.04564438 0.90954816 0.01631036
  0.00718343]
 [0.16404445 0.31871292 0.02297932 0.0778124  0.2728319  0.0269332
  0.11668579]
 [0.09460087 0.07442056 0.04689006 0.18857306 0.29929292 0.25066605
  0.04

INFO:tensorflow:step = 0, loss = 2.565458
INFO:tensorflow:global_step/sec: 34.6945
INFO:tensorflow:step = 100, loss = 1.8014348 (2.865 sec)
INFO:tensorflow:global_step/sec: 46.7577
INFO:tensorflow:step = 200, loss = 1.8234658 (2.139 sec)
INFO:tensorflow:global_step/sec: 45.7854
INFO:tensorflow:step = 300, loss = 1.8681576 (2.185 sec)
INFO:tensorflow:global_step/sec: 45.1662
INFO:tensorflow:step = 400, loss = 1.8065517 (2.215 sec)
INFO:tensorflow:global_step/sec: 46.9101
INFO:tensorflow:step = 500, loss = 1.7688875 (2.130 sec)
INFO:tensorflow:global_step/sec: 48.6162
INFO:tensorflow:step = 600, loss = 1.8091943 (2.057 sec)
INFO:tensorflow:global_step/sec: 47.9795
INFO:tensorflow:step = 700, loss = 1.7552068 (2.084 sec)
INFO:tensorflow:global_step/sec: 49.0611
INFO:tensorflow:step = 800, loss = 1.7460608 (2.039 sec)
INFO:tensorflow:global_step/sec: 49.0325
INFO:tensorflow:step = 900, loss = 1.7895137 (2.039 sec)
INFO:tensorflow:global_step/sec: 48.6459
INFO:tensorflow:step = 1000, loss =

INFO:tensorflow:step = 2000, loss = 1.7103506 (2.068 sec)
INFO:tensorflow:global_step/sec: 44.5561
INFO:tensorflow:step = 2100, loss = 1.7132193 (2.223 sec)
INFO:tensorflow:global_step/sec: 46.8182
INFO:tensorflow:step = 2200, loss = 1.5931551 (2.136 sec)
INFO:tensorflow:global_step/sec: 46.4457
INFO:tensorflow:step = 2300, loss = 1.617738 (2.155 sec)
INFO:tensorflow:global_step/sec: 47.9205
INFO:tensorflow:step = 2400, loss = 1.5616504 (2.085 sec)
INFO:tensorflow:global_step/sec: 49.1998
INFO:tensorflow:step = 2500, loss = 1.6350744 (2.032 sec)
INFO:tensorflow:global_step/sec: 48.8475
INFO:tensorflow:step = 2600, loss = 1.6528076 (2.048 sec)
INFO:tensorflow:global_step/sec: 45.5746
INFO:tensorflow:step = 2700, loss = 1.4510199 (2.194 sec)
INFO:tensorflow:global_step/sec: 46.2114
INFO:tensorflow:step = 2800, loss = 1.7374983 (2.164 sec)
INFO:tensorflow:global_step/sec: 46.9356
INFO:tensorflow:step = 2900, loss = 1.3172494 (2.132 sec)
INFO:tensorflow:global_step/sec: 47.3649
INFO:tensor

INFO:tensorflow:step = 4000, loss = 1.6497427 (2.068 sec)
INFO:tensorflow:global_step/sec: 47.8336
INFO:tensorflow:step = 4100, loss = 1.4552752 (2.069 sec)
INFO:tensorflow:global_step/sec: 50.0235
INFO:tensorflow:step = 4200, loss = 1.7433643 (1.999 sec)
INFO:tensorflow:global_step/sec: 49.2394
INFO:tensorflow:step = 4300, loss = 1.6419775 (2.030 sec)
INFO:tensorflow:global_step/sec: 49.2465
INFO:tensorflow:step = 4400, loss = 1.5646645 (2.031 sec)
INFO:tensorflow:global_step/sec: 48.9739
INFO:tensorflow:step = 4500, loss = 1.3602941 (2.042 sec)
INFO:tensorflow:global_step/sec: 46.9768
INFO:tensorflow:step = 4600, loss = 1.6078621 (2.129 sec)
INFO:tensorflow:global_step/sec: 46.5008
INFO:tensorflow:step = 4700, loss = 1.3796446 (2.150 sec)
INFO:tensorflow:global_step/sec: 45.7423
INFO:tensorflow:step = 4800, loss = 1.2849002 (2.186 sec)
INFO:tensorflow:global_step/sec: 48.2014
INFO:tensorflow:step = 4900, loss = 1.4258271 (2.074 sec)
INFO:tensorflow:global_step/sec: 48.9878
INFO:tenso

KeyboardInterrupt: 

## EVAL

In [16]:
test_df = pd.read_csv("~/data/emotion_reco/test_face_dataset.csv",sep=",")
test_df.rename(columns={"Pixels":"pixels","Emotion":"emotion"},inplace=True)
test_df = test_df.apply(lambda x: string_to_float_list(x),axis=1)

In [17]:
x_test = df_to_tensor(test_df["pixels"])
y_test = test_df["emotion"].values

In [18]:
def evaluate(cpts, clf, x, y):
    for checkpoint in cpts:
        eval_results = clf.evaluate(
            input_fn=lambda:eval_input_fn(x, y, len(y)),
            checkpoint_path=checkpoint)

In [19]:
checks = ["tmp/convnet_dense_model-2/model.ckpt-123974",
"tmp/convnet_dense_model-2/model.ckpt-132837",
"tmp/convnet_dense_model-2/model.ckpt-141690",
"tmp/convnet_dense_model-2/model.ckpt-150545",
"tmp/convnet_dense_model-2/model.ckpt-159385"]

In [20]:
evaluate(checks, emotion_classifier_dense, x_test, y_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-27-22:49:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from tmp/convnet_dense_model-2/model.ckpt-123974


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for tmp/convnet_dense_model-2/model.ckpt-123974
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_18_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/